# Introduction

FU Orionis objects (hereafter FUors) are a group of pre-main-sequence stars that abruptly increase in brightness by several magnitudes, lasting decades or longer. The FUors’ large amplitude bursts are credited to enhanced mass accretion from the surrounding circumstellar disk, with various triggering mechanisms being proposed over the years, e.g. gravitational and/or magnetorotational instabilities, thermal instabilities, or interactions with binary companions. 

This notebook uses data from new 1 mm continuum observations of the FUor HBC722, obtained with the Atacama Large Millimeter/Submillimeter Array (ALMA), to calculate its circumstellar disk mass.

# Import Dependencies

We begin by importing the necessary libraries.

In [7]:
import math
import numpy as np
from astropy import units as u
from astropy.constants import h, k_B, c

# Function Definitions

Then, we define some functions that will be useful to us later on. Specifically, the Planck function:

$$\begin{equation*}
    B(\nu,\;T)=\frac{2h\nu^3}{c^2}\frac{1}{e^{\frac{h\nu}{k_B T}}-1}
\end{equation*}$$

and the total circumstellar disk mass:

$$\begin{equation*}
    M=100\frac{d^2S_\nu}{B_\nu(T_D)\kappa_\nu\quad}
\end{equation*}$$

In [8]:
# Equation for the Planck function
def Planck_function(h, k_B, c, T, nu):
  return (2 * h * nu**3 / c**2) * (1 / (math.exp(h * nu / k_B / T) - 1)) / u.s**4 / u.Hz**4 / u.sr
# Equation for the total disk mass
def disk_mass(r, S, d, K, B):
  return (r * S * d**2 / K / B * u.erg / u.Hz / u.s / u.sr).to(u.solMass)

# Define Constants

Also define some useful constants, including the integrated flux, which can be found in the `data.csv` file. 

In [9]:
# Flux density or Integrated flux [units: erg s^-1 cm^-2 Hz^-1]
S = 0.00292278 * u.Jy.to(u.erg / u.s / u.cm**2 / u.Hz)
# Distance [units: cm]
d = 800 * u.parsec.to(u.cm)
# Gas to dust ratio
r = 100

# Calculate the Planck Function

The Planck function describes the spectral density of electromagnetic radiation emitted by a blackbody in thermal equilibrium at a given temperature $T$, when there is no net flow of matter or energy between the body and its environment.

In [10]:
# Planck constant [units: erg s]
h = h.to(u.erg * u.s)
# Boltzmann constant [units: erg K^-1]
k_B = k_B.to(u.erg / u.K)
# Speed of light [units: cm s^-1]
c = c.to(u.cm / u.s)
# Temperature [units: K]
T = 30 * u.K
# Frequency [units: Hz] 
nu = (233 * u.GHz).to(u.Hz)
# Planck function [units: erg s^-1 cm^-2 Hz^-1 sr^-1]
B = Planck_function(h, k_B, c, T, nu)

# Compute the Opacity

Next, we calculate the dust opacities using data from Table 1 of [Ossenkopf, Henning (1994)](https://ui.adsabs.harvard.edu/abs/1994A&A...291..943O/abstract).

In [11]:
# Wavelength data [units: microns]
x = np.array([1.00e2, 1.17e2, 1.36e2, 1.58e2, 1.85e2, 2.26e2, 3.50e2, 5.00e2, 7.00e2, 1.00e3, 1.30e3])
# Opacity data [units: cm^2 g^-1]
y = np.array([8.65e1, 6.75e1, 5.25e1, 4.09e1, 3.07e1, 2.17e1, 1.01e1, 5.04e0, 2.57e0, 1.37e0, 8.99e-1])

# Total Circumstellar Disk Mass

Finally, we can compute the total circumstellar disk mass for the FUor of interest: HBC722.

In [12]:
# Switching to log scale
x_log = []
y_log = []
for i in range (len(x)):
  x_log.append(math.log10(x[i]))
for j in range (len(y)):
  y_log.append(math.log10(y[j]))

# Fitting best fit line (m: slope, c: y-intercept)
m, c = np.polyfit(x_log, y_log, deg=1)

# Converting our specific wavelength into log scale
wavelength = nu.to(u.micron, equivalencies=u.spectral())
wavelength_log = math.log10(wavelength / u.micron)

# Solving for opacity (in log scale)
K_log = (wavelength_log * m) + c

# Opacity [units: cm^2 g^-1]
K = 10**K_log * u.cm**2 / u.g

## Calculating the total disk mass
# Total mass [units: solar mass]
M = disk_mass(r, S, d, K, B)
print("Total mass of HBC722 disk:", M)

Total mass of HBC722 disk: 0.024082890859563992 solMass


# Conclusions

We conclude that HBC722 has a circumstellar disk of approximately $0.024~\text{M}_{\odot}$, which is in agreement with previous results. 